In [5]:
import dask.dataframe as dd

In [3]:
import logging
import numpy as np
import os
import pandas as pd
import s3fs
import sys
from s3_utils import write_parquet_from_pd

from utils_income.utils_income.commons import common_utils
from utils_income.utils_income.commons.common_utils import to_numeric

import logging
import sys

s3 = s3fs.S3FileSystem()

# Logger SetUp
# -------------------------------------------------------------------------------
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
logFormatter = logging.Formatter(
    "%(asctime)s %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p"
)
handler.setFormatter(logFormatter)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
# -------------------------------------------------------------------------------

# Kikoya

In [55]:
kikoya_df = dd.read_parquet("s3://data-science-kavak-dev/projects/cerberus/v2/dev/income/01_raw/data_folios.parquet").compute()

In [56]:
no_nulos = kikoya_df.net_income_verified.notnull()
kikoya_df = kikoya_df[no_nulos]

In [57]:
filter_list = list(kikoya_df.researchable_id)


# Bureau Sections

In [6]:
path_intl = "s3://data-science-kavak-dev/projects/cerberus/v2/dev/data/kikoya/03_processed/2022-04-26/"

In [54]:
sc_df = dd.read_parquet(path_intl+"scSections.pq")

In [53]:
rs_df = dd.read_parquet(path_intl+"rsSections.pq/")

In [58]:
rs_df= rs_df.loc[rs_df.researchable_id.isin(filter_list)].compute()
rs_df.drop_duplicates(subset="researchable_id",inplace=True, keep="last")
sc_df= sc_df.loc[sc_df.researchable_id.isin(filter_list)].compute()
sc_df.drop_duplicates(subset="researchable_id",inplace=True, keep="last")
df =  pd.merge(rs_df,sc_df, on="researchable_id")

In [86]:
kikoya_df

,net_income_verified,researchable_id,folio
0,10000.0,266606,75255
1,11000.0,264835,73628
3,16000.0,239957,50500
5,30000.0,240508,51008
7,15000.0,263327,72240
...,...,...,...
68847,17500.0,1525752,152551008032022
68848,18000.0,1546587,154634916032022
68849,3000.0,1561266,156102822032022
68851,8000.0,1583657,158341930032022


# Core

In [78]:
file ="s3://data-science-kavak-dev/projects/cerberus/v2/dev/income/00_core_bancario/20220405 Core bancario v22f_MORA AL 31032022.xlsm"
core = pd.read_excel(file, 'Registros', engine='openpyxl', header=8)
core = core[[x for x in core.columns if 'Unna' not in str(x)]].copy()
core = core.dropna(subset=['Consecutivo'])

In [79]:
select_cols = ['Consecutivo',
 'RFC','Estimate', 'BC_Score ','Ingreso ',
 'Dependientes',
 'Calculo PI',
 'Condición de Ingreso',
 'Ing_Disp','WO',
 'FECHA WO',
 'MONTO WO',
 'REFERENCIA WO']
core = core[select_cols]

In [80]:
core['Estimate'].isna().sum()

0

In [85]:
df

,researchable_id,declarativa,fecha_de_apertura_de_la_cuenta_mas_antigua,fecha_de_apertura_de_la_cuenta_mas_reciente,fecha_de_apertura_mas_reciente_de_una_cuenta_en_despacho_de_cobranza_o_administadora_de_cartera,fecha_de_integracion_en_la_base_de_datos,fecha_de_la_consulta_mas_reciente,fecha_de_la_consulta_mas_reciente_realizada_por_un_despacho_de_cobranza_o_administadora_de_cartera,mensaje_de_alerta,moneda_del_credito,...,total_de_saldos_actuales_para_cuentas_de_pagos_fijos_e_hipotecarios,total_de_saldos_actuales_para_cuentas_revolventes_y_sin_limite_pre-establecido,total_de_saldos_vencidos_para_cuentas_de_pagos_fijos_e_hipotecarios,total_de_saldos_vencidos_para_cuentas_revolventes_y_sin_limite_pre-establecido,codigo_de_razon_1,codigo_de_razon_2,codigo_de_razon_3,codigo_del_score,nombre_del_score,valor_del_score
0,265186,No,18082011,23032020,00000000,24122012,28102020,00000000,None,Pesos_Mexicanos,...,000000000+,000206121+,000000000,000000000,15,21,28,Reporte_de_Credito_con_BC_Score,BC SCORE,0682
1,261815,No,14032015,28062018,00000000,26052009,24102020,00000000,None,Pesos_Mexicanos,...,001345460+,000049915+,000000000,000000000,04,None,None,Reporte_de_Credito_con_BC_Score,BC SCORE,0721
2,265137,No,27092011,13042019,00000000,26092011,28102020,00000000,None,Pesos_Mexicanos,...,000013495+,000000000+,000013495,000007543,61,None,None,Reporte_de_Credito_con_BC_Score,BC SCORE,0635
3,261906,No,16022005,13032020,00000000,29052007,30102020,00000000,None,Pesos_Mexicanos,...,000003976+,000004619+,000009541,000000000,21,17,None,Reporte_de_Credito_con_BC_Score,BC SCORE,0580
4,260507,No,20042018,24122019,00000000,25062007,22102020,00000000,None,Pesos_Mexicanos,...,000142849+,000000000+,000018225,000011130,17,18,31,Reporte_de_Credito_con_BC_Score,BC SCORE,0566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589812,322014,No,00000000,00000000,00000000,02082016,08122020,00000000,None,None,...,000000000+,000000000+,000000000,000000000,None,None,None,Reporte_de_Credito_con_BC_Score,BC SCORE,-009
589813,318707,No,15082001,19052015,00000000,09042002,06122020,00000000,None,Pesos_Mexicanos,...,000000000+,000000000+,000000000,000000000,21,04,None,Reporte_de_Credito_con_BC_Score,BC SCORE,0700
589814,320311,No,23042019,23042019,00000000,04092013,07122020,00000000,None,Pesos_Mexicanos,...,000000000+,000007786+,000000000,000000000,51,None,None,Reporte_de_Credito_con_BC_Score,BC SCORE,0704
589815,310650,No,18082005,11112019,00000000,31012017,01122020,00000000,None,Pesos_Mexicanos,...,000000000+,000003370+,000000000,000000000,09,15,None,Reporte_de_Credito_con_BC_Score,BC SCORE,0663


In [84]:
pd.merge(df,core, left_on="estimate",right_on="Estimate",how="inner")

KeyError: 'estimate'

In [82]:
core['Estimate'].str

0         1517280
1         1575118
2         1637040
3         1675358
4         1676838
           ...   
22844    17223322
22845    17290394
22846    17189718
22847    17293904
22848    17296873
Name: Estimate, Length: 22849, dtype: object

In [81]:
import re
core['cleanEstimate'] = core['Estimate'].str.extract(re.compile('(\d+)'), expand=False).astype(int)

ValueError: cannot convert float NaN to integer

In [ ]:
def load_intl(path_intl):
    selected_files = ["rsSections.parquet","scSections.parquet" ]
    df_rs = dd.read_parquet(path_intl+ "rsSections.parquet")


In [9]:
import boto3
import dask.dataframe as dd
import os
import pandas as pd
from s3_utils.s3_utils import write_parquet_from_pd, start_logger


class Kikoya:
    def __init__(self):
        self.bucket = 'kavak-datalake-serving-layer'
        self.prefix = 'data_science/kikoya/ultima_actualizacion/'
        self.basepath = f's3://{self.bucket}/{self.prefix}'
        self.base_keys = ['folio', 'folio_id', 'researchable_id']
        self.connection = self.set_kikoya_client()

    def set_kikoya_client(self):
        sts = boto3.client('sts')
        role = sts.assume_role(
            RoleArn='arn:aws:iam::405458584171:role/dataScienceAccess',
            RoleSessionName="KavakCapital")
        self.creds = {
            "key": role['Credentials']['AccessKeyId'],
            "secret": role['Credentials']['SecretAccessKey'],
            "token": role['Credentials']['SessionToken']
        }

        return boto3.client('s3', aws_access_key_id=self.creds['key'], aws_secret_access_key=self.creds['secret'],
                            aws_session_token=self.creds['token'])


class KikoyaIncome(Kikoya):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tipos_de_objetos = ['product_applications', 
                                 'product_application_users',
                                 'product_application_financial_data']
        self.paths = self.connect_to_files()
        self.save_path = "s3://data-science-kavak-dev/projects/cerberus/v2/dev/income/01_raw/" #clase abstracta raw data

    # TODO class connetor o blue prints?
    def connect_to_files(self, start_date='2020-10'):
        s3_client = self.set_kikoya_client()
        all_bucket_files = s3_client.list_objects(Bucket=self.bucket, Prefix=self.prefix).get('Contents')
        files = [f"s3://{self.bucket}/{x.get('Key')}" for x in all_bucket_files]
        files = pd.Series(files, name='uris').reset_index()
        files['tipo'] = files['uris'].apply(lambda x: os.path.splitext(os.path.split(x)[1])[0])
        files = files.loc[files['tipo'].isin(self.tipos_de_objetos)]
        files['fechas'] = files['uris'].str.extract('(\d{4}\-\d{2})', expand=False)
        files['fechas'] = pd.to_datetime(files['fechas'])
        files = files.loc[files['fechas'] >= f'{start_date}']
        return files[["tipo", "uris"]].groupby('tipo')['uris'].agg(list).to_dict()

    def products_app(self):
        cols_to_product_applications = ['decorated_folio', 'id']
        cols_to_rename_product_applications = {'id': 'folio_id', 'decorated_folio': 'folio'}
        product_applications = dd.read_parquet(self.paths['product_applications'], storage_options=self.creds,
                                               columns=cols_to_product_applications)
        return product_applications.rename(columns=cols_to_rename_product_applications)

    def users_app(self):
        cols_to_product_applications_users = ['product_application_id', 'id']
        cols_to_rename_applications_users = {'product_application_id': 'folio_id', 'id': 'researchable_id'}
        product_applications_users = dd.read_parquet(
            self.paths['product_application_users'],
            storage_options=self.creds,
            columns=cols_to_product_applications_users)
        return product_applications_users.rename(columns=cols_to_rename_applications_users)

    def financial_app(self):
        cols_financial_data = ['researchable_id', 'net_income_verified']
        return dd.read_parquet(self.paths['product_application_financial_data'], storage_options=self.creds,
                               columns=cols_financial_data)

    def make_kk_table(self):
        data_folios = self.products_app().merge(self.users_app(), how='left', on='folio_id')
        data_folios = data_folios.merge(self.financial_app(), how='left', on='researchable_id')
        data_folios = data_folios.drop_duplicates()
        data_folios = data_folios.compute()
        subset_columns = ["net_income_verified", "researchable_id", "folio"]
        data_folios = data_folios[subset_columns]
        data_folios["folio"] = data_folios["folio"].str.replace("KV", "")
        write_parquet_from_pd(self.save_path + "data_folios.parquet", data_folios)

In [10]:
KikoyaIncome().make_kk_table()

,net_income_verified,researchable_id,folio
0,10000.0,266606,75255
1,11000.0,264835,73628
2,NaN,241674,52156
3,16000.0,239957,50500
4,NaN,261833,70842
...,...,...,...
68848,18000.0,1546587,154634916032022
68849,3000.0,1561266,156102822032022
68850,NaN,1527825,152758309032022
68851,8000.0,1583657,158341930032022
